In [7]:
import numpy as np
import matplotlib.pyplot as plt


@np.vectorize
def spikemodel(x_seconds):
    # from VS12, https://github.com/volodymyrss/spiacs-duration/blob/master/spikelike.C
	x = x_seconds*20.
	return 5.5*(0.3429/(np.exp(x/0.253)+np.exp(-1*x/0.1)) + \
		   0.003/(np.exp(x/15.)+np.exp(-1*x/0.1)) + \
		   0.003/(np.exp(x/2.)+np.exp(-1*x/0.1)))


t = np.linspace(-1, 3, 1000)


import integralclient as ic

ic.get_hk('ACS', )

plt.plot(t, spikemodel(t))



failed to get AUTH with env due to: 'K8S_SECRET_INTEGRAL_CLIENT_SECRET'
got AUTH with homefile for integral-limited


TypeError: get_hk() takes 0 positional arguments but 1 was given